In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%matplotlib inline

import numpy as np
import scipy.io as io
from pyDOE import lhs
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

from complexPyTorch.complexLayers import ComplexLinear

import cplxmodule
from cplxmodule import cplx
from cplxmodule.nn import RealToCplx, CplxToReal, CplxSequential, CplxToCplx
from cplxmodule.nn import CplxLinear, CplxModReLU, CplxAdaptiveModReLU, CplxModulus, CplxAngle

# To access the contents of the parent dir
import sys; sys.path.insert(0, '../')
import os
from scipy.io import loadmat
from utils import *
from models import TorchComplexMLP, ImaginaryDimensionAdder, cplx2tensor, ComplexTorchMLP, complex_mse
from preprocess import *

# Model selection
from sparsereg.model import STRidge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge
from pde_diff import TrainSTRidge, FiniteDiff, print_pde
from RegscorePy.bic import bic

# Fancy optimizers
from madgrad import MADGRAD
from lbfgsnew import LBFGSNew

/usr/local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


#### Data preparation

In [2]:
data = pickle_load('../deephpms_data/KS_simple.pkl')
# data = loadmat('../PDE_FIND_experimental_datasets/kuramoto_sivishinky.mat')

t = data['tt'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = np.real(data['uu'])

X_sol, T_sol = np.meshgrid(x, t)

x_star = X_sol.flatten()[:,None]
t_star = T_sol.flatten()[:,None]

X_star = np.hstack((x_star, t_star))
u_star = Exact.T.flatten()[:,None]

# DATA_PATH = '../PDE_FIND_experimental_datasets/kuramoto_sivishinky.mat'
# X, T, Exact = space_time_grid(data_path=DATA_PATH, real_solution=True, uniform=True, x_limit=None, t_limit=None)
# X_star, u_star = get_trainable_data(X, T, Exact)

# Bound
ub = X_star.max(axis=0)
lb = X_star.min(axis=0)

# For identification
N = 10000
idx = np.arange(N)
# idx = np.random.choice(X_star.shape[0], N, replace=False)
X_train = X_star[idx,:]
u_train = u_star[idx,:]

print("Training with", N, "samples")

# Unsup data
include_N_res = False
if include_N_res:
    N_res = N//2
    idx_res = np.array(range(X_star.shape[0]-1))[~idx]
    idx_res = np.random.choice(idx_res.shape[0], N_res, replace=True)
    X_res = X_star[idx_res, :]
    print(f"Training with {N_res} unsup samples")
    X_u_train = np.vstack([X_train, X_res])
    u_train = np.vstack([u_train, torch.rand(X_res.shape[0], 1) - 1000])
    # del X_res
else: print("Not including N_res")
    
# Convert to torch.tensor
X_train = to_tensor(X_train, True)
u_train = to_tensor(u_train, False)
X_star = to_tensor(X_star, True)
u_star = to_tensor(u_star, False)

# lb and ub are used in adversarial training
scaling_factor = 1.0
lb = scaling_factor*to_tensor(lb, False)
ub = scaling_factor*to_tensor(ub, False)

# Feature names
feature_names=('uf', 'u_x', 'u_xx', 'u_xxx', 'u_xxxx')

Loaded from ../deephpms_data/KS_simple.pkl
Training with 10000 samples
Not including N_res


In [5]:
fdc = FinDiffCalculator(X_sol, T_sol, Exact, acc_order=10)

# Even if you use the true vals of dx and dt -> The finite difference is not working with a non-uniform grid
# fdc.dx = 0.09817477042468103
# fdc.dt = 0.4

fd_u_t = to_tensor(fdc.finite_diff(1, diff_order=1), False)[idx, :]
fd_derivatives = fdc.finite_diff_from_feature_names(feature_names)
for d in fd_derivatives: fd_derivatives[d] = to_tensor(fd_derivatives[d], False)

Do not use this class with complex-valued input arrays.
This class applies 1 transpose to the Exact before doing the job.
dx = 0.001953125
dt = 0.4


In [6]:
tmp = None
for f in feature_names:
    print('Concatenating', f)
    if tmp == None: tmp = fd_derivatives[f]
    else: tmp = torch.cat([tmp, fd_derivatives[f]], dim=-1)

fd_derivatives = tmp[idx, :]
(((fd_u_t+fd_derivatives[:, 4:5]+(fd_derivatives[:, 0:1]*fd_derivatives[:, 1:2])+fd_derivatives[:, 2:3]))**2).mean().item()

Concatenating uf
Concatenating u_x
Concatenating u_xx
Concatenating u_xxx
Concatenating u_xxxx


6.97397984753032e+22